In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import ticker
from copy import deepcopy
import time
from scipy.stats import ks_2samp as ks
from scipy.stats import pearsonr
from scipy.stats import spearmanr

# from matplotlib import rc
# rc('font',**{'family':'serif','serif':['Times New Roman'],'size':16})
# rc('text',usetex=True)
# rc('patch',antialiased=False)
# from matplotlib import rcParams
# rcParams['font.size'] = 16.0
# plt.rc('font', size=16)

#import seaborn as sns
import pandas as pd
#import pymc3 as pm
#import theano.tensor as tt

In [ ]:
# OPTIONS

calc_model = 1


# READ

data = pd.read_csv('planetdata_200806.csv') #[SR] changed file location
star_labels = ['hostnames', 'fst_mass','fst_masserr1','fst_masserr2','fst_age','fst_ageerr1','fst_ageerr2','fst_met','fst_meterr1','fst_meterr2','fst_dist','fst_disterr1','fst_disterr2']
planet_labels = ['fpl_letter','fpl_bmasse','fpl_bmasseerr1','fpl_bmasseerr2','fpl_smax','fpl_smaxerr1','fpl_smaxerr2','fpl_orbper','fpl_orbpererr1','fpl_orbpererr2',
                 'fpl_eccen','fpl_eccenerr1','fpl_eccenerr2','fpl_rade','fpl_radeerr1','fpl_radeerr2','fpl_dens','fpl_denserr1','fpl_denserr2']
analysis_labels = ['Nss','Pnull','BIC1','rhoN','Phigh','HJflag','insamp']
stars = deepcopy(data[star_labels])
print("Stars: \n", stars)
# stellar properties + hostnames
planets = deepcopy(data[planet_labels])
print("Planets: \n", planets)
# planet properties
analysis = deepcopy(data[analysis_labels])
print("Analysis: \n", analysis)
# analysis is Pnull etc

Stars: 
      hostnames fst_mass fst_masserr1 fst_masserr2 fst_age fst_ageerr1  \
0       11 Com      2.7          0.3         -0.3       -           -   
1       11 UMi     2.78         0.69        -0.69    1.56        0.54   
2       14 And      2.2          0.1         -0.2     4.5         1.9   
3       14 Her      0.9         0.04        -0.04    5.24           -   
4     16 Cyg B     1.08         0.04        -0.04     7.4         0.1   
...        ...      ...          ...          ...     ...         ...   
1517   tau Gem      2.3          0.3         -0.3    1.22        0.76   
1518   ups And      1.3            -            -     5.0           -   
1519   ups And      1.3            -            -     5.0           -   
1520   ups And      1.3            -            -     5.0           -   
1521    xi Aql      2.2            -            -     7.1         3.6   

     fst_ageerr2 fst_met fst_meterr1 fst_meterr2  fst_dist fst_disterr1  \
0              -   -0.35        0.09   

In [ ]:
# FORMAT

for label in star_labels[1:]:
    stars[label] = pd.to_numeric(stars[label], errors='coerce') # convert stars[label] columns to number
    # pd.to_numeric -> convert argument to a numeric type.
    # errors{‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
    # If ‘coerce’, then invalid parsing will be set as NaN
    if 'err2' in label:
        # if the string "err2" is in the star label, convert to positive
        # TODO: do I need this step?
        stars[label] = -stars[label]
for label in planet_labels[1:]:
    planets[label] = pd.to_numeric(planets[label], errors='coerce')
    if 'err2' in label:
        planets[label] = -planets[label]
for label in analysis_labels[:5]:
    analysis[label] = pd.to_numeric(analysis[label], errors='coerce')

In [ ]:
# FIX OVERDENSITY STATS OF MULTIPLES

# TODO: I don't think I need this section
unique_all, counts_all = np.unique(stars['hostnames'].values, return_counts=True)
multiples = unique_all[counts_all > 1]
for sys in multiples:
    idx = (stars['hostnames'] == sys)
    analysis['Pnull'][idx] = np.mean(analysis['Pnull'][idx])
    analysis['rhoN'][idx] = np.mean(analysis['rhoN'][idx])
    analysis['Phigh'][idx] = np.mean(analysis['Phigh'][idx])

/tmp/ipykernel_60516/1291788700.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['Pnull'][idx] = np.mean(analysis['Pnull'][idx])
/tmp/ipykernel_60516/1291788700.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['rhoN'][idx] = np.mean(analysis['rhoN'][idx])
/tmp/ipykernel_60516/1291788700.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['Phigh'][idx] = np.mean(analysis['Phigh'][idx])


In [ ]:
# SUBSAMPLES

# TODO: don't most of this section
giants = planets['fpl_bmasse'] >= 50.
neptunes = (planets['fpl_bmasse'] < 50.) & (planets['fpl_bmasse'] > 5.)
rockys = planets['fpl_bmasse'] <= 5.
# Uses the line below
low_orig = analysis['Phigh'] < 0.16
high_orig = analysis['Phigh'] > 0.84
ambiguous = (analysis['Phigh'] >= 0.16) & (analysis['Phigh'] <= 0.84)
HJs = analysis['HJflag'] == 'Y'
# Uses the line below
cuts = (analysis['Pnull'] < 0.05) & (analysis['Nss'] >= 400) & (stars['fst_age'] >= 1.) & (stars['fst_age'] <= 4.5) & (stars['fst_mass'] >= 0.7) & (stars['fst_mass'] <= 2.)
masses = ~np.isnan(planets['fpl_bmasseerr1'])
radii = ~np.isnan(planets['fpl_radeerr1'])

unique_all, counts_all = np.unique(stars['hostnames'].values[cuts], return_counts=True)
# Uses the line below
unique_low, counts_low = np.unique(stars['hostnames'].values[cuts & low_orig], return_counts=True)
# np.unique(arr) Find the unique elements of an array. 
# return_counts=True/False If Tue, also return the number of times each unique item appears in ar.
# Returns the sorted unique elements of an array and three optional additional outputs, including the return_counts
unique_high, counts_high = np.unique(stars['hostnames'].values[cuts & high_orig], return_counts=True)

In [ ]:
np_all = np.arange(1, np.max(counts_all)+1)
nsys_all = np.bincount(counts_all)[1:]
np_low = np.arange(1, np.max(counts_low)+1)
nsys_low = np.bincount(counts_low)[1:]
np_high = np.arange(1, np.max(counts_high)+1)
nsys_high = np.bincount(counts_high)[1:]

In [ ]:
closein = planets['fpl_smax'] <= 30.
unique_all30, counts_all30 = np.unique(stars['hostnames'].values[cuts & closein], return_counts=True)
unique_low30, counts_low30 = np.unique(stars['hostnames'].values[cuts & low_orig & closein], return_counts=True)
unique_high30, counts_high30 = np.unique(stars['hostnames'].values[cuts & high_orig & closein], return_counts=True)

In [ ]:
np_all30 = np.arange(1, np.max(counts_all30)+1)
nsys_all30 = np.bincount(counts_all30)[1:]
np_low30 = np.arange(1, np.max(counts_low30)+1)
nsys_low30 = np.bincount(counts_low30)[1:]
np_high30 = np.arange(1, np.max(counts_high30)+1)
nsys_high30 = np.bincount(counts_high30)[1:]

In [ ]:
npair_per = np.zeros_like(np_all)
for i in range(1,np.size(np_all)):
    npair_per[i] = np.math.factorial(np_all[i])/(np.math.factorial(2)*np.math.factorial(np_all[i]-2))
npair_all = np.sum(nsys_all*npair_per)
npair_low = np.sum(nsys_low*npair_per[0:np.max(np_low)])
npair_high = np.sum(nsys_high*npair_per[0:np.max(np_high)])

In [ ]:
subs = [unique_low, unique_high] # TODO: what is subs?
for sname in subs: # TODO: what is sname? Rows
    ns = len(sname) # TODO: what is ns? Number of rows? ie nimber of stars?
    samp = np.array([])
    for i in range(ns):
        samp = np.append(samp, np.where(stars['hostnames'] == sname[i])[0])
    ms_m = np.percentile(stars['fst_mass'][samp],50.)
    ms_0 = ms_m-np.percentile(stars['fst_mass'][samp],16.)
    ms_2 = np.percentile(stars['fst_mass'][samp],84.)-ms_m
    str_m = '$'+str(np.round(ms_m,2))+'_{-'+str(np.round(ms_0,2))+'}^{+'+str(np.round(ms_2,2))+'}$'
    met_m = np.nanpercentile(stars['fst_met'][samp],50.)
    met_0 = met_m-np.nanpercentile(stars['fst_met'][samp],16.)
    met_2 = np.nanpercentile(stars['fst_met'][samp],84.)-met_m
    str_met = '$'+str(np.round(met_m,2))+'_{-'+str(np.round(met_0,2))+'}^{+'+str(np.round(met_2,2))+'}$'
    age_m = np.nanpercentile(stars['fst_age'][samp],50.)
    age_0 = age_m-np.nanpercentile(stars['fst_age'][samp],16.)
    age_2 = np.nanpercentile(stars['fst_age'][samp],84.)-age_m
    str_age = '$'+str(np.round(age_m,1))+'_{-'+str(np.round(age_0,1))+'}^{+'+str(np.round(age_2,1))+'}$'
    d_m = np.percentile(stars['fst_dist'][samp],50.)
    d_0 = d_m-np.percentile(stars['fst_dist'][samp],16.)
    d_2 = np.percentile(stars['fst_dist'][samp],84.)-d_m
    str_d = '$'multiple_low+str(np.round(d_m,0))+'_{-'+str(np.round(d_0,0))+'}^{+'+str(np.round(d_2,0))+'}$'
    str_ntotal = '$'+str(np.size(planets['fpl_radeerr1'][samp]))+'$'
    str_ntransit = '$'+str(np.sum(~np.isnan(planets['fpl_radeerr1'][samp])))+'$'
    str_nrv = '$'+str(np.sum(~np.isnan(planets['fpl_bmasseerr1'][samp])))+'$'
    str_ntotal_sys = '$'+str(ns)+'$'
    str_ntransit_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_radeerr1'][samp])])))+'$'
    str_nrv_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_bmasseerr1'][samp])])))+'$'
    print(str_m+' & '+str_met+' & '+str_age+' & '+str_d+' & '+str_ntotal_sys+' & '+str_ntransit_sys+' & '+str_nrv_sys)

$1.09_{-0.24}^{+0.41}$ & $0.08_{-0.28}^{+0.18}$ & $3.2_{-1.3}^{+0.6}$ & $65.0_{-33.0}^{+197.0}$ & $48$ & $17$ & $40$
$1.2_{-0.25}^{+0.21}$ & $0.09_{-0.13}^{+0.15}$ & $2.7_{-1.0}^{+1.3}$ & $218.0_{-167.0}^{+214.0}$ & $253$ & $160$ & $182$


In [ ]:
multiple_all = unique_all[counts_all > 1]
multiple_low = unique_low[counts_low > 1]
# multiple low are the stars that meet the cuts with repeated entries (multiople observed planets in orbit)
multiple_high = unique_high[counts_high > 1]

In [ ]:
subs = [multiple_low, multiple_high]
for sname in subs:
    ns = len(sname)
    samp = np.array([])
    for i in range(ns):
        samp = np.append(samp, np.where(stars['hostnames'] == sname[i])[0])
    ms_m = np.percentile(stars['fst_mass'][samp],50.)
    ms_0 = ms_m-np.percentile(stars['fst_mass'][samp],16.)
    ms_2 = np.percentile(stars['fst_mass'][samp],84.)-ms_m
    str_m = '$'+str(np.round(ms_m,2))+'_{-'+str(np.round(ms_0,2))+'}^{+'+str(np.round(ms_2,2))+'}$'
    met_m = np.nanpercentile(stars['fst_met'][samp],50.)
    met_0 = met_m-np.nanpercentile(stars['fst_met'][samp],16.)
    met_2 = np.nanpercentile(stars['fst_met'][samp],84.)-met_m
    str_met = '$'+str(np.round(met_m,2))+'_{-'+str(np.round(met_0,2))+'}^{+'+str(np.round(met_2,2))+'}$'
    age_m = np.nanpercentile(stars['fst_age'][samp],50.)
    age_0 = age_m-np.nanpercentile(stars['fst_age'][samp],16.)
    age_2 = np.nanpercentile(stars['fst_age'][samp],84.)-age_m
    str_age = '$'+str(np.round(age_m,1))+'_{-'+str(np.round(age_0,1))+'}^{+'+str(np.round(age_2,1))+'}$'
    d_m = np.percentile(stars['fst_dist'][samp],50.)
    d_0 = d_m-np.percentile(stars['fst_dist'][samp],16.)
    d_2 = np.percentile(stars['fst_dist'][samp],84.)-d_m
    str_d = '$'+str(np.round(d_m,0))+'_{-'+str(np.round(d_0,0))+'}^{+'+str(np.round(d_2,0))+'}$'
    str_ntotal = '$'+str(np.size(planets['fpl_radeerr1'][samp]))+'$'
    str_ntransit = '$'+str(np.sum(~np.isnan(planets['fpl_radeerr1'][samp])))+'$'
    str_nrv = '$'+str(np.sum(~np.isnan(planets['fpl_bmasseerr1'][samp])))+'$'
    str_ntotal_sys = '$'+str(ns)+'$'
    str_ntransit_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_radeerr1'][samp])])))+'$'
    str_nrv_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_bmasseerr1'][samp])])))+'$'
    print(str_m+' & '+str_met+' & '+str_age+' & '+str_d+' & '+str_ntotal_sys+' & '+str_ntransit_sys+' & '+str_nrv_sys)

$1.04_{-0.19}^{+0.35}$ & $0.1_{-0.3}^{+0.18}$ & $3.4_{-0.9}^{+0.5}$ & $63.0_{-31.0}^{+144.0}$ & $13$ & $5$ & $12$
$1.15_{-0.2}^{+0.2}$ & $0.07_{-0.14}^{+0.14}$ & $3.0_{-1.2}^{+1.3}$ & $245.0_{-186.0}^{+237.0}$ & $48$ & $32$ & $27$


In [ ]:
multiple = np.concatenate([multiple_low,multiple_high])
ns = len(multiple)
samp = np.array([])
for i in range(ns):
    samp = np.append(samp, np.where(stars['hostnames'] == multiple[i])[0])
masserr = np.nanmean(0.5*(stars['fst_masserr1'][samp]+stars['fst_masserr2'][samp]))
meterr = np.nanmean(0.5*(stars['fst_meterr1'][samp]+stars['fst_meterr2'][samp]))
ageerr = np.nanmean(0.5*(stars['fst_ageerr1'][samp]+stars['fst_ageerr2'][samp]))
disterr = np.nanmean(0.5*(stars['fst_disterr1'][samp]+stars['fst_disterr2'][samp]))
print(masserr, meterr, ageerr, disterr)

0.07902439024390244 0.07222321428571428 1.0739222222222222 2.6584722222222217


In [21]:
###########The relevant bits are lines 186-216 and 242-254.
rosetta = pd.read_csv('KOI-Kepler.csv') #[SR] changed file location
for i in range(len(rosetta)):
    rosetta['Name'][i] = np.char.split(rosetta['Name'].values.astype('str'))[i][0]
    # makes the rosetta Name just the string section of the original name
    rosetta['KOI'][i] = rosetta['KOI'][i][:-3]
    # makes the rosetta KOI the first aprt of the number. Drops the .0? characters
    
# Drop duplicated stars:
rosetta.drop_duplicates(subset=['Name'], inplace=True)
print("Rosetta:\n", rosetta)

multiple_low_K = np.array([])
multiple_low_KOI = np.array([])

print(len(multiple_low))

for i in range(len(multiple_low)):
    #If it starts with 'Kep'
    if multiple_low[i][0:3] == 'Kep':
        # Put that multiple_low entry in the array multiple_low_K
        multiple_low_K = np.append(multiple_low_K, multiple_low[i])
        # Put X in multiple_low_KOI
        # where the values in the 'Name' column of rosetta are the same as the entries in multiple_low, put them in multiple_low_KOI
        multiple_low_KOI = np.append(multiple_low_KOI, rosetta['KOI'].values[np.where(rosetta['Name'].values == multiple_low[i])])

multiple_high_K = np.array([])
multiple_high_KOI = np.array([])
for i in range(len(multiple_high)):
    if multiple_high[i][0:3] == 'Kep':
        multiple_high_K = np.append(multiple_high_K, multiple_high[i])
        multiple_high_KOI = np.append(multiple_high_KOI, rosetta['KOI'].values[np.where(rosetta['Name'].values == multiple_high[i])])

Rosetta:
              Name     KOI
0        Kepler-1  K00001
1       Kepler-10  K00072
3      Kepler-100  K00041
6     Kepler-1000  K01888
7     Kepler-1001  K01889
...           ...     ...
2355   Kepler-995  K01881
2356   Kepler-996  K01882
2357   Kepler-997  K01883
2358   Kepler-998  K01885
2359   Kepler-999  K01886

[1679 rows x 2 columns]
13


In [ ]:
print('')
cks = pd.read_table('Fulton_stars.txt', sep=' ') #[SR] changed file location. This is the CKS star data
subs = [multiple_low_K, multiple_high_K]
subs_KOI = [multiple_low_KOI, multiple_high_KOI]
for j,sname in enumerate(subs):
    sname_KOI = subs_KOI[j]
    ns = len(sname) # TODO: what is this?
    samp = np.array([])
    samp_KOI = np.array([])
    for i in range(ns): # for all characters in star name
        match = np.where(cks['KOI'] == sname_KOI[i])[0]
        if np.size(match) > 0:
            samp = np.append(samp, np.where(stars['hostnames'] == sname[i])[0])
            ############
    ms_m = np.percentile(stars['fst_mass'][samp],50.)
    ms_0 = ms_m-np.percentile(stars['fst_mass'][samp],16.)
    ms_2 = np.percentile(stars['fst_mass'][samp],84.)-ms_m
    str_m = '$'+str(np.round(ms_m,2))+'_{-'+str(np.round(ms_0,2))+'}^{+'+str(np.round(ms_2,2))+'}$'
    met_m = np.nanpercentile(stars['fst_met'][samp],50.)
    met_0 = met_m-np.nanpercentile(stars['fst_met'][samp],16.)
    met_2 = np.nanpercentile(stars['fst_met'][samp],84.)-met_m
    str_met = '$'+str(np.round(met_m,2))+'_{-'+str(np.round(met_0,2))+'}^{+'+str(np.round(met_2,2))+'}$'
    age_m = np.nanpercentile(stars['fst_age'][samp],50.)
    age_0 = age_m-np.nanpercentile(stars['fst_age'][samp],16.)
    age_2 = np.nanpercentile(stars['fst_age'][samp],84.)-age_m
    str_age = '$'+str(np.round(age_m,1))+'_{-'+str(np.round(age_0,1))+'}^{+'+str(np.round(age_2,1))+'}$'
    d_m = np.percentile(stars['fst_dist'][samp],50.)
    d_0 = d_m-np.percentile(stars['fst_dist'][samp],16.)
    d_2 = np.percentile(stars['fst_dist'][samp],84.)-d_m
    str_d = '$'+str(np.round(d_m,0))+'_{-'+str(np.round(d_0,0))+'}^{+'+str(np.round(d_2,0))+'}$'
    str_ntotal = '$'+str(np.size(planets['fpl_radeerr1'][samp]))+'$'
    str_ntransit = '$'+str(np.sum(~np.isnan(planets['fpl_radeerr1'][samp])))+'$'
    str_nrv = '$'+str(np.sum(~np.isnan(planets['fpl_bmasseerr1'][samp])))+'$'
    str_ntotal_sys = '$'+str(len(np.unique(stars['hostnames'][samp])))+'$'
    str_ntransit_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_radeerr1'][samp])])))+'$'
    str_nrv_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_bmasseerr1'][samp])])))+'$'
    print(str_m+' & '+str_met+' & '+str_age+' & '+str_d+' & '+str_ntotal_sys+' & '+str_ntransit_sys+' & '+str_nrv_sys)

multiple_high_K:  ['Kepler-105' 'Kepler-107' 'Kepler-110' 'Kepler-126' 'Kepler-127'
 'Kepler-128' 'Kepler-131' 'Kepler-134' 'Kepler-137' 'Kepler-140'
 'Kepler-141' 'Kepler-144' 'Kepler-145' 'Kepler-314' 'Kepler-321'
 'Kepler-334' 'Kepler-348' 'Kepler-367' 'Kepler-381' 'Kepler-402'
 'Kepler-431' 'Kepler-450' 'Kepler-466' 'Kepler-50' 'Kepler-539'
 'Kepler-65']
multiple_high_KOI:  ['K00115' 'K00117' 'K00124' 'K00260' 'K00271' 'K00274' 'K00283' 'K00295'
 'K00313' 'K00327' 'K00338' 'K00369' 'K00370' 'K01692' 'K01809' 'K01909'
 'K02011' 'K02173' 'K02352' 'K02722' 'K03097' 'K00279' 'K00112' 'K00262'
 'K00372' 'K00085']

$1.01_{-0.16}^{+0.0}$ & $-0.2_{-0.0}^{+0.47}$ & $3.7_{-0.0}^{+0.5}$ & $405.0_{-173.0}^{+0.0}$ & $2$ & $2$ & $2$
$1.15_{-0.13}^{+0.1}$ & $0.04_{-0.07}^{+0.13}$ & $3.3_{-0.9}^{+1.0}$ & $389.0_{-120.0}^{+187.0}$ & $26$ & $25$ & $7$


In [20]:
############################The relevant bits are lines 186-216 and 242-254.
print('')
subs = [multiple_low_K, multiple_high_K]
# subs is made up of two arrays, multiple_low kepler names and multiple_high kepler names
print("subs: \n", subs)
subs_KOI = [multiple_low_KOI, multiple_high_KOI]
# subs_KOI is made up of two arrays, multiple_low_KOI star KOI's and multiple_high_KOI star KOI's
print("subs_KOI: \n", subs_KOI)
for j,sname in enumerate(subs):
    # j is array index (array 0, array 1)
    # sname is a list of star names within each array
    print("j: ", j)
    print("sname: ", sname)
    sname_KOI = subs_KOI[j]
    # sname_KOI is the star's KOI
    print("sname_KOI: ", sname_KOI)
    ns = len(sname) # the number of stars/star names in the array j
    print("ns: ", ns)
    samp = np.array([])
    samp_KOI = np.array([])
    # For 0-number of star names in array
    for i in range(ns):
        match = np.where(cks['KOI'] == sname_KOI[i])[0]
        # match is a list of indicies where the entry in the 'KOI' column of cks is the same as the ith entry in sname_KOI (array of star KOIs)
        
        # If there are some entries in match (i.e. there are entries in cks["KOI"] that are the same as those in sname_KOI):
        if np.size(match) > 0:
            match_NASA = np.where(stars['hostnames'] == sname[i])[0]
            # match_NASA are the indicies only (where.()[0]) where the 'hostnames' in stars (NASA_EA) are the same as those in sname (list of star names e.g. Kepler-111)
            # stars = deepcopy(data[star_labels])
            samp = np.append(samp, match_NASA) # addmatch_NASA to the samp array
            samp_KOI = np.append(samp_KOI, np.repeat(match[0], np.size(match_NASA))).flatten()
    #####################
    ms_m = np.percentile(cks['mass'][samp_KOI],50.)
    ms_0 = ms_m-np.percentile(cks['mass'][samp_KOI],16.)
    ms_2 = np.percentile(cks['mass'][samp_KOI],84.)-ms_m
    str_m = '$'+str(np.round(ms_m,2))+'_{-'+str(np.round(ms_0,2))+'}^{+'+str(np.round(ms_2,2))+'}$'
    met_m = np.nanpercentile(cks['FeH'][samp_KOI],50.)
    met_0 = met_m-np.nanpercentile(cks['FeH'][samp_KOI],16.)
    met_2 = np.nanpercentile(cks['FeH'][samp_KOI],84.)-met_m
    str_met = '$'+str(np.round(met_m,2))+'_{-'+str(np.round(met_0,2))+'}^{+'+str(np.round(met_2,2))+'}$'
    age_m = np.nanpercentile(10.**(cks['age'][samp_KOI]-9.),50.)
    age_0 = age_m-np.nanpercentile(10.**(cks['age'][samp_KOI]-9.),16.)
    age_2 = np.nanpercentile(10.**(cks['age'][samp_KOI]-9.),84.)-age_m
    str_age = '$'+str(np.round(age_m,1))+'_{-'+str(np.round(age_0,1))+'}^{+'+str(np.round(age_2,1))+'}$'
    d_m = np.percentile(stars['fst_dist'][samp],50.)
    d_0 = d_m-np.percentile(stars['fst_dist'][samp],16.)
    d_2 = np.percentile(stars['fst_dist'][samp],84.)-d_m
    str_d = '$'+str(np.round(d_m,0))+'_{-'+str(np.round(d_0,0))+'}^{+'+str(np.round(d_2,0))+'}$'
    str_ntotal = '$'+str(np.size(planets['fpl_radeerr1'][samp]))+'$'
    str_ntransit = '$'+str(np.sum(~np.isnan(planets['fpl_radeerr1'][samp])))+'$'
    str_nrv = '$'+str(np.sum(~np.isnan(planets['fpl_bmasseerr1'][samp])))+'$'
    str_ntotal_sys = '$'+str(len(np.unique(stars['hostnames'][samp])))+'$'
    str_ntransit_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_radeerr1'][samp])])))+'$'
    str_nrv_sys = '$'+str(np.size(np.unique(stars['hostnames'][samp][~np.isnan(planets['fpl_bmasseerr1'][samp])])))+'$'
    print(str_m+' & '+str_met+' & '+str_age+' & '+str_d+' & '+str_ntotal_sys+' & '+str_ntransit_sys+' & '+str_nrv_sys)

#quit()


subs: 
 [array(['Kepler-104', 'Kepler-378', 'Kepler-454'], dtype='<U32'), array(['Kepler-105', 'Kepler-107', 'Kepler-110', 'Kepler-126',
       'Kepler-127', 'Kepler-128', 'Kepler-131', 'Kepler-134',
       'Kepler-137', 'Kepler-140', 'Kepler-141', 'Kepler-144',
       'Kepler-145', 'Kepler-314', 'Kepler-321', 'Kepler-334',
       'Kepler-348', 'Kepler-367', 'Kepler-381', 'Kepler-402',
       'Kepler-431', 'Kepler-450', 'Kepler-466', 'Kepler-50',
       'Kepler-539', 'Kepler-65'], dtype='<U32')]
subs_KOI: 
 [array(['K00111', 'K02287', 'K00273'], dtype=object), array(['K00115', 'K00117', 'K00124', 'K00260', 'K00271', 'K00274',
       'K00283', 'K00295', 'K00313', 'K00327', 'K00338', 'K00369',
       'K00370', 'K01692', 'K01809', 'K01909', 'K02011', 'K02173',
       'K02352', 'K02722', 'K03097', 'K00279', 'K00112', 'K00262',
       'K00372', 'K00085'], dtype=object)]
j:  0
sname:  ['Kepler-104' 'Kepler-378' 'Kepler-454']
sname_KOI:  ['K00111' 'K02287' 'K00273']
ns:  3
$0.82_{-0.0}^{+0.2